# Sequence to Sequence model to extract summary,objective from resume (input: entire resume)


## 207 resumes


In [18]:
from __future__ import print_function
import csv,os
import collections
from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=5
epochs=50
latent_dim=128
data_path="/home/santhosh/resumes_folder/custom_annotator/annotator-server/static/files/Data_Tracter_Resumes_in_TXT/csv"

## vectorizing the data

In [19]:
input_resumes=[]
output_summary=[]
input_tokens=collections.Counter()
output_tokens=set()
files=os.listdir(data_path)
count=0
for file in files[:20]:
    with open(data_path+'/'+file,'r') as csv_file:
        reader=csv.reader(csv_file)
        input_text=" "
        output_text=" "
        count=0
        for Input_text,output in reader:
            Input_text=Input_text.strip().lower()
            input_text+=Input_text+' \n'
            if(output=='1'):
                output_text+=Input_text+' \n'
            
            if count==15:
                break
            count+=1
            
    """
        print("*"*50)
        print(" "*50)
        #print(input_text)
        print("-"*50)
        print(output_text)
        print(" "*50)
        print("*"*50)
    if count==2:
        break
    count+=1
    """

    # We use "<SOL>" as the "start sequence" character
    # for the targets, and "<EOL>" as "end sequence" character.
    output_text='<SOL> '+output_text+' <EOL>'
    input_resumes.append(input_text)
    output_summary.append(output_text)
    for word in input_text.split():
        if word not in input_tokens:
            input_tokens[word]+=1
    for word in output_text.split():
        if word not in output_tokens:
            output_tokens.add(word)


In [20]:
print(len(input_tokens))
num_encoder_tokens=min(len(input_tokens),3000)
input_tokens=[word for word,count in input_tokens.most_common(num_encoder_tokens-1)]
input_tokens=sorted(list(input_tokens))
output_tokens=sorted(list(output_tokens))
num_decoder_tokens=len(output_tokens)
max_encoder_seq_len=max([len(text.split()) for text in input_resumes])
max_decoder_seq_len=max([len(text.split()) for text in output_summary])

print('Number of samples:',len(input_resumes))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


1000
Number of samples: 20
number of unique input token: 1000
number of unique output token: 725
Max Sequence length for inputs: 144
Max Sequence length for outputs: 132


## defining token2index

In [21]:
input_token2index=dict([(word,i) for i,word in enumerate(input_tokens)])
output_token2index=dict([(word,i) for i,word in enumerate(output_tokens)])
input_token2index['UNK']=num_encoder_tokens-1

## defing encoder_input,decoder_input and decoder_output

In [22]:
encoder_input_data=np.zeros((len(input_resumes),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')


## creating training dataset

In [23]:
for i,(input_text,target_text) in enumerate(zip(input_resumes,output_summary)):
    for t,word in enumerate(input_text.split()[:max_encoder_seq_len]):
        if word not in input_token2index:
            word="UNK"
        encoder_input_data[i,t,input_token2index[word]]=1
        
    for t,word in enumerate(target_text.split()):
        decoder_input_data[i,t,output_token2index[word]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,output_token2index[word]]=1


## Define an input sequence and process it.

In [24]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


## Set up the decoder, using encoder_states as initial state.

In [25]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [26]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 1000)   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 725)    0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 128), (None, 578048      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 128),  437248      input_6[0][0]                    
                                                                 lstm_3[0][1]                     
          

## function to test model

In [27]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_word_index = dict(
    (i, word) for word, i in input_token2index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in output_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token2index['<SOL>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<EOL>' or
           len(decoded_sentence.split()) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    if('<EOL>' in decoded_sentence):
        decoded_sentence=" ".join(decoded_sentence.split()[:-1])
    return decoded_sentence


## Run training

In [ ]:

iteration=0
"""
# load weights
print('loading the weights')
model=load_model('resume_level.h5')

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)
for i in range(1):
    index=np.random.randint(len(input_resumes))
    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=decode_sequence(encoded_input_sequence)
    print("-"*50)
    print(input_resumes[index])
    print(" "*50)
    print("*"*50+"\nOUTPUT"+" "*50)
    print(output_sequence)
    print("-"*50+"\n"+" "*50)
"""
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/iteration_resume_level.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="resume_level_checkpoints.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    #prepare sample_data to test 5 samples:
    print("-"*50)
    for i in range(1):
        index=np.random.randint(len(input_resumes))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print("-"*50)
        print(input_resumes[index])
        print(" "*50)
        print("*"*50+"\nOUTPUT"+" "*50)
        print(output_sequence)
        
        
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('resume_level.h5')

file_data, iteration:23
Iteration: 24
Train on 16 samples, validate on 4 samples
Epoch 1/50
16/16 [==============================] - 8s 526ms/step - loss: 4.3153 - acc: 0.0303 - val_loss: 3.7508 - val_acc: 0.0284


/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/50
16/16 [==============================] - 5s 339ms/step - loss: 4.1113 - acc: 0.0388 - val_loss: 3.6498 - val_acc: 0.0265
Epoch 3/50
16/16 [==============================] - 6s 350ms/step - loss: 3.8508 - acc: 0.0379 - val_loss: 3.7322 - val_acc: 0.0265
Epoch 4/50
16/16 [==============================] - 6s 355ms/step - loss: 3.8020 - acc: 0.0388 - val_loss: 3.7306 - val_acc: 0.0284
Epoch 5/50
16/16 [==============================] - 6s 374ms/step - loss: 3.7715 - acc: 0.0407 - val_loss: 3.7777 - val_acc: 0.0284
Epoch 6/50
16/16 [==============================] - 6s 348ms/step - loss: 3.7558 - acc: 0.0431 - val_loss: 3.7703 - val_acc: 0.0303
Epoch 7/50
16/16 [==============================] - 6s 345ms/step - loss: 3.7364 - acc: 0.0431 - val_loss: 3.7959 - val_acc: 0.0360
Epoch 8/50
16/16 [==============================] - 6s 370ms/step - loss: 3.7194 - acc: 0.0417 - val_loss: 3.9039 - val_acc: 0.0341
Epoch 9/50
16/16 [==============================] - 6s 360ms/step - loss: 3.

16/16 [==============================] - 6s 358ms/step - loss: 3.2666 - acc: 0.0507 - val_loss: 4.8302 - val_acc: 0.0303
Epoch 3/50
16/16 [==============================] - 6s 389ms/step - loss: 3.2350 - acc: 0.0492 - val_loss: 4.8896 - val_acc: 0.0379
Epoch 4/50
16/16 [==============================] - 6s 383ms/step - loss: 3.2189 - acc: 0.0497 - val_loss: 4.8238 - val_acc: 0.0341
Epoch 5/50
16/16 [==============================] - 7s 410ms/step - loss: 3.2101 - acc: 0.0549 - val_loss: 5.0031 - val_acc: 0.0341
Epoch 6/50
16/16 [==============================] - 6s 368ms/step - loss: 3.2270 - acc: 0.0492 - val_loss: 4.9486 - val_acc: 0.0341
Epoch 7/50
16/16 [==============================] - 6s 364ms/step - loss: 3.2363 - acc: 0.0488 - val_loss: 5.1603 - val_acc: 0.0265
Epoch 8/50
16/16 [==============================] - 5s 342ms/step - loss: 3.1988 - acc: 0.0526 - val_loss: 5.0313 - val_acc: 0.0227
Epoch 9/50
16/16 [==============================] - 7s 415ms/step - loss: 3.2000 - acc:

Epoch 4/50
16/16 [==============================] - 6s 352ms/step - loss: 2.7041 - acc: 0.0857 - val_loss: 5.3575 - val_acc: 0.0360
Epoch 5/50
16/16 [==============================] - 6s 352ms/step - loss: 2.7453 - acc: 0.0762 - val_loss: 5.3107 - val_acc: 0.0379
Epoch 6/50
16/16 [==============================] - 6s 371ms/step - loss: 2.6637 - acc: 0.0862 - val_loss: 5.3368 - val_acc: 0.0398
Epoch 7/50
16/16 [==============================] - 6s 383ms/step - loss: 2.6578 - acc: 0.0843 - val_loss: 5.2677 - val_acc: 0.0379
Epoch 8/50
16/16 [==============================] - 6s 377ms/step - loss: 2.6592 - acc: 0.0824 - val_loss: 5.3381 - val_acc: 0.0436
Epoch 9/50
16/16 [==============================] - 6s 382ms/step - loss: 2.7308 - acc: 0.0795 - val_loss: 5.2956 - val_acc: 0.0360
Epoch 10/50
16/16 [==============================] - 6s 397ms/step - loss: 2.6860 - acc: 0.0848 - val_loss: 5.3494 - val_acc: 0.0246
Epoch 11/50
16/16 [==============================] - 6s 348ms/step - loss: 

...

16/16 [==============================] - 6s 370ms/step - loss: 0.0575 - acc: 0.6515 - val_loss: 5.7802 - val_acc: 0.0455
Epoch 4/50
16/16 [==============================] - 6s 380ms/step - loss: 0.0406 - acc: 0.6529 - val_loss: 5.7769 - val_acc: 0.0455
Epoch 5/50
16/16 [==============================] - 6s 346ms/step - loss: 0.0388 - acc: 0.6548 - val_loss: 5.7811 - val_acc: 0.0436
Epoch 6/50
16/16 [==============================] - 6s 376ms/step - loss: 0.0366 - acc: 0.6544 - val_loss: 5.7899 - val_acc: 0.0436
Epoch 7/50
16/16 [==============================] - 6s 391ms/step - loss: 0.0356 - acc: 0.6539 - val_loss: 5.8226 - val_acc: 0.0436
Epoch 8/50
16/16 [==============================] - 6s 404ms/step - loss: 0.0413 - acc: 0.6544 - val_loss: 5.8251 - val_acc: 0.0398
Epoch 9/50
16/16 [==============================] - 6s 378ms/step - loss: 0.0352 - acc: 0.6562 - val_loss: 5.8503 - val_acc: 0.0398
Epoch 10/50
16/16 [==============================] - 6s 365ms/step - loss: 0.0361 - acc

16/16 [==============================] - 6s 364ms/step - loss: 0.0193 - acc: 0.6553 - val_loss: 5.9295 - val_acc: 0.0436
Epoch 2/50
16/16 [==============================] - 6s 348ms/step - loss: 0.0191 - acc: 0.6558 - val_loss: 5.9925 - val_acc: 0.0473
Epoch 3/50
16/16 [==============================] - 6s 377ms/step - loss: 0.0132 - acc: 0.6562 - val_loss: 6.0026 - val_acc: 0.0360
Epoch 4/50
16/16 [==============================] - 6s 389ms/step - loss: 0.0117 - acc: 0.6553 - val_loss: 6.0602 - val_acc: 0.0436
Epoch 5/50
16/16 [==============================] - 6s 357ms/step - loss: 0.0121 - acc: 0.6553 - val_loss: 6.0141 - val_acc: 0.0379
Epoch 6/50
16/16 [==============================] - 6s 357ms/step - loss: 0.0109 - acc: 0.6558 - val_loss: 6.0470 - val_acc: 0.0417
Epoch 7/50
16/16 [==============================] - 6s 372ms/step - loss: 0.0111 - acc: 0.6553 - val_loss: 5.9677 - val_acc: 0.0511
Epoch 8/50
16/16 [==============================] - 5s 328ms/step - loss: 0.0115 - acc:

16/16 [==============================] - 6s 346ms/step - loss: 0.0049 - acc: 0.6572 - val_loss: 6.1042 - val_acc: 0.0511
Epoch 3/50
16/16 [==============================] - 6s 381ms/step - loss: 0.0039 - acc: 0.6572 - val_loss: 6.1075 - val_acc: 0.0530
Epoch 4/50
16/16 [==============================] - 6s 373ms/step - loss: 0.0045 - acc: 0.6572 - val_loss: 6.0330 - val_acc: 0.0436
Epoch 5/50
16/16 [==============================] - 7s 417ms/step - loss: 0.0085 - acc: 0.6548 - val_loss: 6.0376 - val_acc: 0.0473
Epoch 6/50
16/16 [==============================] - 7s 412ms/step - loss: 0.0041 - acc: 0.6562 - val_loss: 6.0657 - val_acc: 0.0492
Epoch 7/50
16/16 [==============================] - 6s 373ms/step - loss: 0.0033 - acc: 0.6558 - val_loss: 6.1422 - val_acc: 0.0417
Epoch 8/50
16/16 [==============================] - 6s 347ms/step - loss: 0.0030 - acc: 0.6558 - val_loss: 6.1362 - val_acc: 0.0455
Epoch 9/50
16/16 [==============================] - 6s 387ms/step - loss: 0.0029 - acc:

16/16 [==============================] - 7s 409ms/step - loss: 9.9807e-04 - acc: 0.6558 - val_loss: 6.1681 - val_acc: 0.0530
Epoch 4/50
16/16 [==============================] - 6s 392ms/step - loss: 9.3555e-04 - acc: 0.6558 - val_loss: 6.1593 - val_acc: 0.0549
Epoch 5/50
16/16 [==============================] - 6s 391ms/step - loss: 8.9819e-04 - acc: 0.6558 - val_loss: 6.1431 - val_acc: 0.0492
Epoch 6/50
16/16 [==============================] - 6s 385ms/step - loss: 0.0010 - acc: 0.6558 - val_loss: 6.1667 - val_acc: 0.0511
Epoch 7/50
16/16 [==============================] - 6s 367ms/step - loss: 8.6706e-04 - acc: 0.6558 - val_loss: 6.1658 - val_acc: 0.0530
Epoch 8/50
16/16 [==============================] - 6s 358ms/step - loss: 8.2055e-04 - acc: 0.6558 - val_loss: 6.1361 - val_acc: 0.0473
Epoch 9/50
16/16 [==============================] - 6s 377ms/step - loss: 8.1787e-04 - acc: 0.6558 - val_loss: 6.2994 - val_acc: 0.0473
Epoch 10/50
16/16 [==============================] - 6s 400ms/s

16/16 [==============================] - 6s 387ms/step - loss: 8.9618e-04 - acc: 0.6558 - val_loss: 6.2575 - val_acc: 0.0492
Epoch 5/50
16/16 [==============================] - 6s 376ms/step - loss: 7.3185e-04 - acc: 0.6558 - val_loss: 6.2621 - val_acc: 0.0473
Epoch 6/50
16/16 [==============================] - 6s 393ms/step - loss: 6.4673e-04 - acc: 0.6558 - val_loss: 6.2727 - val_acc: 0.0473
Epoch 7/50
16/16 [==============================] - 6s 371ms/step - loss: 5.8804e-04 - acc: 0.6558 - val_loss: 6.2788 - val_acc: 0.0473
Epoch 8/50
16/16 [==============================] - 6s 386ms/step - loss: 5.4179e-04 - acc: 0.6558 - val_loss: 6.2935 - val_acc: 0.0473
Epoch 9/50
16/16 [==============================] - 6s 359ms/step - loss: 4.9215e-04 - acc: 0.6558 - val_loss: 6.3006 - val_acc: 0.0455
Epoch 10/50
16/16 [==============================] - 6s 375ms/step - loss: 4.5825e-04 - acc: 0.6558 - val_loss: 6.3095 - val_acc: 0.0455
Epoch 11/50
16/16 [==============================] - 6s 37